In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
# load data
df_train = pd.read_csv("/content/drive/My Drive/airbnb-recruiting-new-user-bookings/procesed_train_users.csv")
df_test = pd.read_csv("/content/drive/My Drive/airbnb-recruiting-new-user-bookings/processed_test_users.csv")
df_sessions = pd.read_csv("/content/drive/My Drive/airbnb-recruiting-new-user-bookings/sessions.csv")

In [0]:
df_train.head()

,id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,tfa_time,dac_pred_diff,dac_season,dac_weekday,tfa_pred_diff,tfa_season,tfa_weekday,tfa_trange
0,gxn3p5htnn,FEMALE,37.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,6,28,2009,3,19,4,5,summer,Monday,6,spring,Thursday,dawn
1,820tgsjxq7,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2011,5,25,2009,5,23,17,4,spring,Wednesday,6,spring,Saturday,afternoon
2,4ft3gnwmtx,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2010,9,28,2009,6,9,23,5,fall,Tuesday,6,summer,Tuesday,night
3,bjjt8pjhuk,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2011,12,5,2009,10,31,6,4,winter,Monday,6,fall,Saturday,dawn
4,87mebub9p4,MALE,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,9,14,2009,12,8,6,5,fall,Tuesday,6,winter,Tuesday,dawn


In [0]:
df_sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [0]:
# 1. action_type은 drop <-missing value 많으므로
# 2. action_detail은 drop <-missing value 많으므로
df_sessions=df_sessions.drop(['action_type', 'action_detail'], axis = 1)

In [0]:
df_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567737 entries, 0 to 10567736
Data columns (total 4 columns):
user_id         object
action          object
device_type     object
secs_elapsed    float64
dtypes: float64(1), object(3)
memory usage: 322.5+ MB


In [0]:
df_sessions.head()

,user_id,action,device_type,secs_elapsed
0,d1mm9tcy42,lookup,Windows Desktop,319.0
1,d1mm9tcy42,search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,Windows Desktop,301.0
3,d1mm9tcy42,search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,Windows Desktop,435.0


In [0]:
# 다른 column도 missing value 확인 
u_action=df_sessions['action'].unique() # 확인 결과 nan이라는 missing value 존재 -> 특별한 처리를 거치지는 않고 개수에는 포함, 종류에서는 무시
u_device=df_sessions['device_type'].unique() # '-unknown-'이라는 missing value 존재 -> unknown device도 하나의 종류로 인정

In [0]:
df_sessions['secs_elapsed'].isnull() # missing value 존재 -> imputation

0           False
1           False
2           False
3           False
4           False
5           False
6           False
7           False
8           False
9           False
10          False
11          False
12          False
13          False
14          False
15          False
16          False
17          False
18          False
19          False
20          False
21          False
22          False
23          False
24          False
25          False
26          False
27          False
28          False
29          False
            ...  
10567707    False
10567708    False
10567709    False
10567710    False
10567711     True
10567712    False
10567713    False
10567714    False
10567715    False
10567716    False
10567717    False
10567718    False
10567719    False
10567720    False
10567721    False
10567722    False
10567723    False
10567724    False
10567725    False
10567726     True
10567727    False
10567728    False
10567729    False
10567730    False
10567731  

In [0]:
# train set의 user들의 session id
train_sessions_id=list(df_train['id'].unique())
# test set의 user들의 session id
test_sessions_id=list(df_test['id'].unique())

In [0]:
# train set session과 test set session 분리
df_train_sessions=df_sessions.loc[df_sessions['user_id'].isin(train_sessions_id)]
df_test_sessions=df_sessions.loc[df_sessions['user_id'].isin(test_sessions_id)]

In [0]:
# secs_elapsed missing value imputation by average
av = df_train_sessions.secs_elapsed.values
print(df_train_sessions.secs_elapsed.describe())
mean_secs=int(df_train_sessions.secs_elapsed.mean())

count    5.464142e+06
mean     2.045577e+04
std      9.260970e+04
min      0.000000e+00
25%      2.360000e+02
50%      1.137000e+03
75%      8.493000e+03
max      1.799977e+06
Name: secs_elapsed, dtype: float64


In [0]:
## 일괄적으로 missing secs_elapsed impute
df_train_sessions.secs_elapsed=df_train_sessions.secs_elapsed.fillna(value=mean_secs)
df_test_sessions.secs_elapsed=df_test_sessions.secs_elapsed.fillna(value=mean_secs)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [0]:
df_train_sessions['action']=df_train_sessions['action'].fillna(value='-unknown-') 
df_test_sessions['action']=df_test_sessions['action'].fillna(value='-unknown-') 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
df_train['action_sum']=0
df_test['action_sum']=0
df_train['action_types']=0
df_test['action_types']=0
df_train['device_types']=0
df_test['device_types']=0
df_train['total_secs']=0
df_test['total_secs']=0
df_train['mean_secs']=0
df_test['mean_secs']=0

In [0]:
df_train.loc[df_train['id']=='d1mm9tcy42']

,id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,tfa_time,dac_pred_diff,dac_season,dac_weekday,tfa_pred_diff,tfa_season,tfa_weekday,tfa_trange,action_sum,action_types,device_types,total_secs,mean_secs
137021,d1mm9tcy42,MALE,62.0,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,2014,1,1,2014,1,1,0,1,winter,Wednesday,1,winter,Wednesday,morning,0,0,0,0,0


In [0]:
df_train_sessions.loc[df_train_sessions['user_id']=='d1mm9tcy42', 'secs_elapsed'].values.sum()

3447984.0

In [0]:
for id in train_sessions_id:
  unique_action=[]
  unique_device=[]
  
  # 3. user가 실행한 action 개수<-action
  action_sum=df_train_sessions.loc[df_train_sessions['user_id']==id].count()
  df_train.loc[df_train['id']==id,'action_sum']=action_sum
  
  # 4. user가 실행한 action 가지수<-action <- '-unknown-' 무시
  unique_action=list(df_train_sessions.loc[df_train_sessions['user_id']==id, 'action'].unique())
  if('-unknown-' in unique_action):
    unique_action.remove('-unknown-')
  df_train.loc[df_train['id']==id,'action_types']=len(unique_action)
  
  # 5. 사용한 device_type 가지수
  unique_device=list(df_train_sessions.loc[df_train_sessions['user_id']==id, 'device_type'].unique())
  df_train.loc[df_train['id']==id,'device_types']=len(unique_device)
  
  # 6. secs_elapsed  총합
  total_secs=df_train_sessions.loc[df_train_sessions['user_id']==id, 'secs_elapsed'].values.sum()
  df_train.loc[df_train['id']==id,'total_secs']=total_secs

  # 7. 6. secs_elapsed 총합 / 1. action 개수 ->secs_elapsed 평균 
  df_train.loc[df_train['id']==id,'mean_secs']=total_secs / action_sum
  

KeyboardInterrupt: ignored

In [0]:
# 8. device_type 중 가장 많이 사용한 device type
'''코드: 모든 [id ,'device_type'] 들을 list()화=all_device
	list 중 가장 count 수 많은 것 선택
	'mode_device'
	all_device=list(df_train.loc[df_train['id']==id, 'device_type'].values)
	mode=0
	for item in all_device:
		if(all_device.count(item)>mode):
			mode=all_device.count(item)
			mode_item_idx=all_device.index(item)
	df_train.loc[df_train['id']==id, 'mode_device']=all_device[mode_item_idx]'''
# 9. device_type unique한 종류 중 각각 사용한 횟수
'''코드: 먼저 for문 밖에 df_sessions['device_type'].unique()를 확인, 모든 item에 대해 df_train['']=0 생성
	unique_device for문 돌면서 all_device.count(item)을 df_train[,item]에 저장
	for item in unique_device:'''